## Indice


1. **Obtención de Datos de las Recetas**
   - [1.1 Obtención de la Información Nutricional](#11-obtención-de-la-información-nutricional)
   - [1.2 Obtención del Resto de Datos](#12-obtención-del-resto-de-información)
2. **Generador del Menú de Comidas**

   - [2. Generador del Menú de Comidas](#2-generador-del-menú-de-comidas)

3. **Mostrar el Menú con la Información Completa**
   - [3. Mostrar el Menú con la Información Completa](#3-mostrar-el-menú-con-la-información-completa)


### **1. OBTENCIÓN DE DATOS DE LAS RECETAS**


Para personalizar las dietas a cada persona con la mayor eficiencia posible, se
va a usar la **API de Spoonacular**.


En primer lugar, hay que obtener una base de datos de todas las recetas, incluyendo
la siguiente información:

- ID de la receta
- Nombre de la receta
- Ingredientes
- Instrucciones de preparación
- Información nutricional (calorías, proteínas, carbohidratos y grasas)


Para ello, debemos hacer dos llamadas a la API: una para sacar la información
nutricional, y otra para obtener los ingredientes, las
instrucciones de preparación.

> La URL a la primera llamada es: https://api.spoonacular.com/recipes/findByNutrients

> La URL a la segunda llamada es: https://api.spoonacular.com/recipes/complexSearch


El código que se muestra a continuación permite obtener la base de datos descrita
anteriormente en formato JSON:


In [1]:
import requests
import json
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

URL_NUTRIENTS = "https://api.spoonacular.com/recipes/findByNutrients"
URL_SEARCH = "https://api.spoonacular.com/recipes/complexSearch"
URL_PLANNER = "https://api.spoonacular.com/mealplanner/generate"

API_KEY1 = os.getenv("API_KEY1")
API_KEY2 = os.getenv("API_KEY2")
API_KEY3 = os.getenv("API_KEY3")
API_KEY4 = os.getenv("API_KEY4")
API_KEY5 = os.getenv("API_KEY5")


MAX_RESULTS_PER_REQUEST = 100  # Máximo permitido por la API

In [3]:
def get_total_recipes():
    params = {
        "apiKey": API_KEY1,
        "number": 1
    }
    response = requests.get(URL_SEARCH, params=params)
    if response.status_code == 200:
        data = response.json()
        return data.get("totalResults", 0)  # Número total de recetas
    else:
        print(f"Error al obtener total: {response.status_code} - {response.text}")
        return 0

get_total_recipes()

5230

#### **1.1 OBTENCIÓN DE LA INFORMACIÓN NUTRICIONAL**


In [4]:
def get_recipes_nutrition():
    total_recipes = 5230
    if total_recipes == 0:
        return []
    
    recipes = []
    for offset in range(0, total_recipes, MAX_RESULTS_PER_REQUEST):
        params = {
            "apiKey": API_KEY1,
            "minCarbs": 0,
            "maxCarbs": 500,
            "minProtein": 0,
            "maxProtein": 500,
            "minCalories": 1,
            "maxCalories": 2000,
            "minFat": 0,
            "maxFat": 500,
            "number": MAX_RESULTS_PER_REQUEST,
            "offset": offset
        }
        response = requests.get(URL_NUTRIENTS, params=params)
        if response.status_code == 200:
            data = response.json()
            recipes.extend(data)
            print(f"Descargadas {len(recipes)} recetas de {total_recipes}...")
        else:
            print(f"Error en la solicitud: {response.status_code} - {response.text}")
            break
    
    return recipes

all_recipes = get_recipes_nutrition()
print(f"\nTotal de recetas obtenidas: {len(all_recipes)}")

Descargadas 100 recetas de 5230...
Descargadas 200 recetas de 5230...
Descargadas 300 recetas de 5230...
Descargadas 400 recetas de 5230...
Descargadas 500 recetas de 5230...
Descargadas 600 recetas de 5230...
Descargadas 700 recetas de 5230...
Descargadas 800 recetas de 5230...
Descargadas 900 recetas de 5230...
Descargadas 1000 recetas de 5230...
Descargadas 1100 recetas de 5230...
Descargadas 1200 recetas de 5230...
Descargadas 1300 recetas de 5230...
Descargadas 1400 recetas de 5230...
Descargadas 1500 recetas de 5230...
Descargadas 1600 recetas de 5230...
Descargadas 1700 recetas de 5230...
Descargadas 1800 recetas de 5230...
Descargadas 1900 recetas de 5230...
Descargadas 2000 recetas de 5230...
Descargadas 2100 recetas de 5230...
Descargadas 2200 recetas de 5230...
Descargadas 2300 recetas de 5230...
Descargadas 2400 recetas de 5230...
Descargadas 2500 recetas de 5230...
Descargadas 2600 recetas de 5230...
Descargadas 2700 recetas de 5230...
Descargadas 2800 recetas de 5230...
D

Una vez obtenidos los datos, hay que hacer un **análisis exploratorio** de estos
para conocerlos en profundidad y poder trabajar mejor. Para ello, se ha optado
por usar pandas como herramienta.


In [21]:
claves = [recipe['id'] for recipe in all_recipes]
len(set(claves)), len(claves)

(1090, 5300)

In [51]:
import pandas as pd

recipes = pd.DataFrame(all_recipes)
recipes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5300 entries, 0 to 5299
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         5300 non-null   int64 
 1   title      5300 non-null   object
 2   image      5300 non-null   object
 3   imageType  5300 non-null   object
 4   calories   5300 non-null   int64 
 5   protein    5300 non-null   object
 6   fat        5300 non-null   object
 7   carbs      5300 non-null   object
dtypes: int64(2), object(6)
memory usage: 331.4+ KB


In [52]:
recipes.head()

,id,title,image,imageType,calories,protein,fat,carbs
0,149425,Herb and Cheddar Cordon Bleu,https://img.spoonacular.com/recipes/149425-312...,jpg,661,68g,21g,43g
1,157081,Omega-3 Creamy Leek Soup,https://img.spoonacular.com/recipes/157081-312...,jpg,766,48g,54g,18g
2,631892,A Classic Strawberry Shortcake,https://img.spoonacular.com/recipes/631892-312...,jpg,514,7g,24g,69g
3,632243,Alouette Crumbled Feta Mediterranean Caponata,https://img.spoonacular.com/recipes/632243-312...,jpg,175,5g,14g,5g
4,632502,Apple Cheddar Turkey Burgers With Chipotle Yog...,https://img.spoonacular.com/recipes/632502-312...,jpg,362,38g,15g,19g


In [53]:
recipes['id'].value_counts()

id
631868    44
665041    44
663500    44
662218    44
660313    44
          ..
637919     1
638375     1
639062     1
639306     1
635310     1
Name: count, Length: 1090, dtype: int64

In [54]:
recipes[recipes['id'] == 631868].sample(10)

,id,title,image,imageType,calories,protein,fat,carbs
1903,631868,4 Ingredient Chicken Pot Pie,https://img.spoonacular.com/recipes/631868-312...,jpg,802,79g,30g,44g
5103,631868,4 Ingredient Chicken Pot Pie,https://img.spoonacular.com/recipes/631868-312...,jpg,802,79g,30g,44g
4403,631868,4 Ingredient Chicken Pot Pie,https://img.spoonacular.com/recipes/631868-312...,jpg,802,79g,30g,44g
2203,631868,4 Ingredient Chicken Pot Pie,https://img.spoonacular.com/recipes/631868-312...,jpg,802,79g,30g,44g
4203,631868,4 Ingredient Chicken Pot Pie,https://img.spoonacular.com/recipes/631868-312...,jpg,802,79g,30g,44g
4103,631868,4 Ingredient Chicken Pot Pie,https://img.spoonacular.com/recipes/631868-312...,jpg,802,79g,30g,44g
2503,631868,4 Ingredient Chicken Pot Pie,https://img.spoonacular.com/recipes/631868-312...,jpg,802,79g,30g,44g
1303,631868,4 Ingredient Chicken Pot Pie,https://img.spoonacular.com/recipes/631868-312...,jpg,802,79g,30g,44g
3003,631868,4 Ingredient Chicken Pot Pie,https://img.spoonacular.com/recipes/631868-312...,jpg,802,79g,30g,44g
3803,631868,4 Ingredient Chicken Pot Pie,https://img.spoonacular.com/recipes/631868-312...,jpg,802,79g,30g,44g


En efecto, hay recetas duplicadas, por lo que hay que eliminar todos los duplicados


In [55]:
recipes_final = recipes.drop_duplicates(subset='id', keep='first')
recipes_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1090 entries, 0 to 1099
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         1090 non-null   int64 
 1   title      1090 non-null   object
 2   image      1090 non-null   object
 3   imageType  1090 non-null   object
 4   calories   1090 non-null   int64 
 5   protein    1090 non-null   object
 6   fat        1090 non-null   object
 7   carbs      1090 non-null   object
dtypes: int64(2), object(6)
memory usage: 76.6+ KB


In [ ]:
def change_to_numeric(dataframe: pd.DataFrame, columns: list) -> pd.DataFrame:
    for column in columns:
        dataframe[column] = dataframe[column].str.replace("g", "")
        dataframe[column] = pd.to_numeric(dataframe[column], errors='coerce')
    return dataframe

columns = ['protein', 'fat', 'carbs']
recipes_final = change_to_numeric(recipes_final, columns)        

In [58]:
recipes_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1090 entries, 0 to 1099
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         1090 non-null   int64 
 1   title      1090 non-null   object
 2   image      1090 non-null   object
 3   imageType  1090 non-null   object
 4   calories   1090 non-null   int64 
 5   protein    1090 non-null   int64 
 6   fat        1090 non-null   int64 
 7   carbs      1090 non-null   int64 
dtypes: int64(5), object(3)
memory usage: 76.6+ KB


In [60]:
recipes_final.to_json("recipes_nutrition.json", orient="records")
recipes_final.to_csv("recipes_nutrition.csv", index=False)

#### **1.2 OBTENCIÓN DEL RESTO DE DATOS**


In [47]:
def filter_data(data: dict[str, list | int]) -> list:
    raw_data = data.get("results", [])
    filtered_data = []
    
    for recipe in raw_data:
        id_ = recipe.get("id", 0)
        price_per_serving = recipe.get("pricePerServing", 0)
        ingredients = recipe.get("extendedIngredients", [])
        ingredients = [
            ingredient.get("original", "") for ingredient in ingredients
        ]
        steps = recipe.get("analyzedInstructions", [])
        steps = steps[0].get("steps", []) if steps else []
        steps = [
            f"{i + 1}. {step.get('step', '')}" for i, step in enumerate(steps)
        ]
        # steps = " ".join(steps)
        
        filtered_data.append( {
            "id": id_,
            "price_per_serving": price_per_serving,
            "ingredients": ingredients,
            "steps": steps
        } )
    
    return filtered_data

In [100]:
def get_recipes_others():
    total_recipes = 5230
    if total_recipes == 0:
        return []
    
    recipes = []
    for offset in range(0, total_recipes, MAX_RESULTS_PER_REQUEST):
        params = {
            "apiKey": API_KEY3,
            "fillIngredients": True,
            "instructionsRequired": True,
            "addRecipeInformation": True,
            "minCarbs": 0,
            "number": MAX_RESULTS_PER_REQUEST,
            "offset": offset
        }
        response = requests.get(URL_SEARCH, params=params)
        if response.status_code == 200:
            data = response.json()
            filtered_data = filter_data(data)
            recipes.extend(filtered_data)
            print(f"Descargadas {len(recipes)} recetas de {total_recipes}...")
        else:
            print(f"Error en la solicitud: {response.status_code} - {response.text}")
            break
    
    return recipes

all_recipes = get_recipes_others()
print(f"\nTotal de recetas obtenidas: {len(all_recipes)}")

Descargadas 100 recetas de 5230...
Descargadas 200 recetas de 5230...
Descargadas 300 recetas de 5230...
Descargadas 400 recetas de 5230...
Descargadas 500 recetas de 5230...
Descargadas 600 recetas de 5230...
Descargadas 700 recetas de 5230...
Descargadas 800 recetas de 5230...
Descargadas 900 recetas de 5230...
Descargadas 1000 recetas de 5230...
Descargadas 1100 recetas de 5230...
Descargadas 1200 recetas de 5230...
Descargadas 1300 recetas de 5230...
Descargadas 1400 recetas de 5230...
Descargadas 1500 recetas de 5230...
Error en la solicitud: 402 - {"status":"failure", "code":402,"message":"Your daily points limit of 150 has been reached. Please upgrade your plan to continue using the API."}

Total de recetas obtenidas: 1500


In [101]:
all_recipes[:3]

[{'id': 715415,
  'price_per_serving': 300.45,
  'ingredients': 'additional toppings: diced avocado, micro greens, chopped basil), 3 medium carrots, peeled and diced, 3 celery stalks, diced, 2 cups fully-cooked chicken breast, shredded (may be omitted for a vegetarian version), ½ cup flat leaf Italian parsley, chopped (plus extra for garnish), 6 cloves of garlic, finely minced, 2 tablespoons olive oil, 28 ounce-can plum tomatoes, drained and rinsed, chopped, 2 cups dried red lentils, rinsed, salt and black pepper, to taste, 1 large turnip, peeled and diced, 8 cups vegetable stock, 1 medium yellow onion, diced',
  'steps': ['1. To a large dutch oven or soup pot, heat the olive oil over medium heat.',
   '2. Add the onion, carrots and celery and cook for 8-10 minutes or until tender, stirring occasionally.',
   '3. Add the garlic and cook for an additional 2 minutes, or until fragrant. Season conservatively with a pinch of salt and black pepper.To the pot, add the tomatoes, turnip and re

Al igual que con los datos anteriores, hay que hacer un análisis exploratorio
de estos datos.


In [102]:
import pandas as pd

In [103]:
others = pd.DataFrame(all_recipes)
others.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 1500 non-null   int64  
 1   price_per_serving  1500 non-null   float64
 2   ingredients        1500 non-null   object 
 3   steps              1500 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 47.0+ KB


In [104]:
others.sample(5)

,id,price_per_serving,ingredients,steps
1332,639510,210.88,"2 red grapefruit, 2 naval oranges, 1 bundle (a...","[1. Using a sharp knife, cut the ends off one..."
463,660292,113.74,"3 Carrots, Diced, 2 Zucchini, Cut up, 1 Onion,...","[1. First up, add your veggies into your slow ..."
1400,640982,135.39,"2 T. olive oil, 1 green bell pepper, chopped, ...",[1. Heat oven to 350 degrees f. In a large pot...
665,657359,115.28,"1 tablespoon of brown sugar, 1 cup of canned p...",[1. Add all ingredients to the blender in orde...
614,644192,204.04,"1 bunch of fresh basil, 5 cloves of garlic – c...",[1. In a large frying pan heat the olive oil a...


In [105]:
others['id'].value_counts()

id
642834     6
673422     6
663545     6
664699     6
1157762    6
          ..
662294     1
1095773    1
662665     1
648995     1
635834     1
Name: count, Length: 1000, dtype: int64

In [106]:
others.drop_duplicates(subset='id', keep='first', inplace=True)
others.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 1000 non-null   int64  
 1   price_per_serving  1000 non-null   float64
 2   ingredients        1000 non-null   object 
 3   steps              1000 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 39.1+ KB


In [107]:
others['price_per_serving'] = others['price_per_serving'] / 100 # To USD
others['price_per_serving']

0      3.0045
1      1.7837
2      0.6909
3      2.7041
4      1.6843
        ...  
995    2.9793
996    0.9500
997    2.9886
998    3.9070
999    1.9623
Name: price_per_serving, Length: 1000, dtype: float64

In [108]:
nutrition = pd.read_json("recipes_nutrition.json")
nutrition.head()

,id,title,image,imageType,calories,protein,fat,carbs
0,149425,Herb and Cheddar Cordon Bleu,https://img.spoonacular.com/recipes/149425-312...,jpg,661,68,21,43
1,157081,Omega-3 Creamy Leek Soup,https://img.spoonacular.com/recipes/157081-312...,jpg,766,48,54,18
2,631892,A Classic Strawberry Shortcake,https://img.spoonacular.com/recipes/631892-312...,jpg,514,7,24,69
3,632243,Alouette Crumbled Feta Mediterranean Caponata,https://img.spoonacular.com/recipes/632243-312...,jpg,175,5,14,5
4,632502,Apple Cheddar Turkey Burgers With Chipotle Yog...,https://img.spoonacular.com/recipes/632502-312...,jpg,362,38,15,19


In [78]:
nutrition.drop(columns=['imageType'], inplace=True)

In [109]:
completed_recipes = pd.merge(nutrition, others, on='id', how='inner')

In [110]:
completed_recipes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215 entries, 0 to 214
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 215 non-null    int64  
 1   title              215 non-null    object 
 2   image              215 non-null    object 
 3   imageType          215 non-null    object 
 4   calories           215 non-null    int64  
 5   protein            215 non-null    int64  
 6   fat                215 non-null    int64  
 7   carbs              215 non-null    int64  
 8   price_per_serving  215 non-null    float64
 9   ingredients        215 non-null    object 
 10  steps              215 non-null    object 
dtypes: float64(1), int64(5), object(5)
memory usage: 18.6+ KB


In [111]:
completed_recipes.drop(columns=['imageType'], inplace=True)

In [112]:
completed_recipes.head()

,id,title,image,calories,protein,fat,carbs,price_per_serving,ingredients,steps
0,149425,Herb and Cheddar Cordon Bleu,https://img.spoonacular.com/recipes/149425-312...,661,68,21,43,3.7369,"1/2 cup breadcrumbs, 2 slices of cheddar chees...",[1. Pre-heat the oven to 350 degrees F (about ...
1,157081,Omega-3 Creamy Leek Soup,https://img.spoonacular.com/recipes/157081-312...,766,48,54,18,5.9514,"250 ml cream, Fresh dill, Juice of 1/2 lemon, ...",[1. Slice the white part of the leek into thin...
2,636361,Brussels Sprout With Mustard And Honey,https://img.spoonacular.com/recipes/636361-312...,272,8,14,25,2.3806,"350 grams Brussels sprout, 1 Fresh chilli, sli...",[1. Wash the brussels sprouts well. Trim the s...
3,636787,Caldo Verde - Portuguese Kale Soup,https://img.spoonacular.com/recipes/636787-312...,493,20,10,70,2.2409,"1/2 large onion chopped, 3 carrots peeled and ...","[1. Chop your onions, slice your carrots and s..."
4,641973,Easy Garlic Roast Leg Of Lamb With Rosemary an...,https://img.spoonacular.com/recipes/641973-312...,496,59,27,1,5.2985,"1 bone-in leg of lamb (about 8 lbs), 5 garlic ...","[1. With the tip of a small sharp knife, poke ..."


In [113]:
completed_recipes.to_json("recipes_completed.json", orient="records")
completed_recipes.to_csv("recipes_completed.csv", index=False)

Una vez obtenida esa base de datos, podemos hacer uso de la API para calcular un
menú diario o semanal de tres comidas por día (desayuno, comida y cena), ajustado
a las necesidades de cada usuario.

> La URL a este recurso es: https://api.spoonacular.com/mealplanner/generate


**2. FUNCIÓN PARA CALCULAR CALORÍAS DEPENDIENDO DEL OBJETIVO (GANAR MÚSCULO, PERDER GRASA O MANTENERSE)**

In [24]:
def calculate_calories(
    gender: str, age: int, weight: float, height: float, exercise: str
    ) -> float:
    """
    Calcula las calorías diarias recomendadas para una persona.
    
    Args:
        gender: hombre o mujer
        age: edad en años
        weight: peso en kg
        height: altura en cm
        exercise: nivel de actividad física (
            sedentario, ligero, moderado, muy activo, extra activo, profesional
            )
    
    Returns:
        total_calories: calorías diarias recomendadas totales para mantener el 
        peso
    """
    if gender.lower() in ['man', 'hombre', 'chico']:
        rmb = (
            (10 * weight) + (6.25 * height) - (5 * age) + 5
        )
    elif gender.lower() in ['woman', 'mujer', 'chica']:
        rmb = (
            (10 * weight) + (6.25 * height) - (5 * age) - 161
        )
    else:
        raise ValueError("Género no válido")
    
    if exercise.lower() in ['sedentary', 'sedentario']:
        total_calories = rmb * 1.2
    elif exercise.lower() in ['lightly active', 'actividad ligera']:
        total_calories = rmb * 1.375
    elif exercise.lower() in ['moderately active', 'actividad moderada']:
        total_calories = rmb * 1.55
    elif exercise.lower() in ['very active', 'muy activo']:
        total_calories = rmb * 1.725
    elif exercise.lower() in ['extra active', 'extra activo']:
        total_calories = rmb * 2
    elif exercise.lower() in ['proffesional', 'profesional']:
        total_calories = rmb * 2.4
    else:
        raise ValueError("Nivel de actividad no válido")
    
    return total_calories

**3. GENERAR PLANIFICADOR DE COMIDAS**

In [98]:
def meal_planner(
    time: str = 'week', 
    calories: float = 2000, 
    diet: str = 'vegetarian', 
    exlude: str = None
) -> dict:
    """
    Planificador de comidas.
    
    Args:
        time: tiempo para planificar (week, day)
        calories: calorías diarias recomendadas
        diet: tipo de dieta (vegan, vegetarian, gluten free, ...)
    
    Returns:
        meal_plan: plan de comidas
    """
    
    params = {
        "apiKey": API_KEY3,
        "timeFrame": time.lower(),
        "targetCalories": calories,
        "diet": diet.lower(),
        "exclude": exlude
    }
    
    response = requests.get(URL_PLANNER, params=params)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Error en la solicitud: {response.status_code} - {response.text}")
        return {}

In [99]:
meal_planner()

{'week': {'monday': {'meals': [{'id': 658624,
     'imageType': 'jpg',
     'title': 'Roasted Plum Oatmeal',
     'readyInMinutes': 45,
     'servings': 4,
     'sourceUrl': 'https://spoonacular.com/roasted-plum-oatmeal-658624'},
    {'id': 642777,
     'imageType': 'jpg',
     'title': 'Fig and Goat Cheese Pizza With Pesto',
     'readyInMinutes': 15,
     'servings': 6,
     'sourceUrl': 'https://spoonacular.com/fig-and-goat-cheese-pizza-with-pesto-642777'},
    {'id': 663225,
     'imageType': 'jpg',
     'title': 'The Best Of England Salad',
     'readyInMinutes': 45,
     'servings': 1,
     'sourceUrl': 'https://spoonacular.com/the-best-of-england-salad-663225'}],
   'nutrients': {'calories': 2000.01,
    'protein': 64.29,
    'fat': 125.76,
    'carbohydrates': 164.76}},
  'tuesday': {'meals': [{'id': 648632,
     'imageType': 'jpg',
     'title': "Jules' Banana Bread",
     'readyInMinutes': 45,
     'servings': 4,
     'sourceUrl': 'https://spoonacular.com/jules-banana-bread-6